In [14]:
from selenium import webdriver
import time
import xlsxwriter
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
driver = webdriver.Chrome('C:\\Users\\ZAHRA\\Downloads\\chromedriver_win32\\chromedriver')
# time.sleep(20)
# links = browser.find_elements_by_xpath('//*[@id="zone-twilight1"]/section/div[2]/div[1]')
# print(len(links))

C:\Users\ZAHRA\AppData\Local\Temp/ipykernel_27588/883510529.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('C:\\Users\\ZAHRA\\Downloads\\chromedriver_win32\\chromedriver')


In [15]:
driver.get("https://www.nytimes.com/section/sports")
time.sleep(2)  # Allow 2 seconds for the web page to open
speed=5
current_scroll_position, new_height= 0, 1
while current_scroll_position <= new_height:
    current_scroll_position += speed
    driver.execute_script("window.scrollTo(0, {});".format(current_scroll_position))
    new_height = driver.execute_script("return document.body.scrollHeight")

from bs4 import BeautifulSoup
from urllib.parse import urljoin
urls = []
soup = BeautifulSoup(driver.page_source, "html.parser")
for parent in soup.find_all(class_="css-112uytv"):
    a_tag = parent.find("a")
    base = "https://www.nytimes.com/"
    link = a_tag.attrs['href']
    url = urljoin(base, link)
    urls.append(url)

In [16]:
len(urls)

80

In [17]:
urls1= urls

errors=[]
dataframe = pd.DataFrame(columns=['article','summary'])
for u in urls1:
    print(u)
    driver.get(u)
    time.sleep(5)
    try:
        try:
            if(driver.find_element("xpath",'//*[@id="story"]/div[3]/p')):
                summary= driver.find_element("xpath",'//*[@id="story"]/div[3]/p').text
        except:
            try:
                summary= driver.find_element("xpath",'//*[@id="fullBleedHeaderContent"]/header/div[2]/div/p[2]').text
            except:
                try:
                    summary= driver.find_element("xpath",'//*[@id="article-summary"]').text
                except:
                    print('no summary')
        divs=  driver.find_elements("xpath",'//*[@id="story"]/section/div')
        i=1
        j=1
        article=''
        for d in divs:
            p_tags=d.find_elements("xpath",'//*[@id="story"]/section/div['+str(j)+']/div/p')
            for p in p_tags:
                article += p.text
                i+=1
            j+=1
        ser = pd.Series([article,summary], index = ['article','summary']) 
        dataframe = dataframe.append(ser, ignore_index=True)
    except:
        print('error in u')
        errors.append(u)
        continue

https://www.nytimes.com/2022/12/06/magazine/ironman-xc.html
no summary
https://www.nytimes.com/2022/12/05/business/nike-kyrie-irving.html
https://www.nytimes.com/2022/12/05/sports/tennis/nick-bollettieri-dead.html
https://www.nytimes.com/2022/12/04/sports/werner-franke-dead.html
https://www.nytimes.com/2022/12/04/sports/mbappe-france-world-cup-bracket.html
https://www.nytimes.com/2022/12/04/sports/soccer/at-qatars-church-city-sunday-comes-on-friday.html
https://www.nytimes.com/2022/12/03/sports/ncaafootball/deion-sanders-colorado-jackson-state.html
https://www.nytimes.com/2022/12/03/sports/ncaafootball/tcu-college-football-playoff.html
https://www.nytimes.com/2022/12/03/sports/soccer/messi-argentina-world-cup.html
https://www.nytimes.com/2022/12/03/sports/soccer/world-cup-tyler-adams-hometown.html
https://www.nytimes.com/2022/12/03/sports/soccer/team-usa-world-cup.html
https://www.nytimes.com/2022/12/03/sports/soccer/us-out-world-cup-netherlands.html
https://www.nytimes.com/2022/12/03/

In [18]:
dataframe

,article,summary
0,To hear more audio stories from publications l...,"The annual event, which returns after a two-ye..."
1,Nike and the N.B.A. star Kyrie Irving ended th...,The sportswear giant suspended its partnership...
2,"Nick Bollettieri, who never competed in elite ...","At his training camps, he employed a tough sty..."
3,"Werner Franke, an esteemed molecular biologist...","In the 1990s, he and his wife unearthed docume..."
4,"DOHA, Qatar — After he scored yet another goal...",France is trying to become just the third nati...
...,...,...
75,Follow live coverage of Brazil vs. South Korea...,Coach Gareth Southgate was unruffled by a scor...
76,Follow live coverage of Morocco vs. Spain and ...,A feisty 0-0 tie against World Cup contender E...
77,Follow live coverage of Brazil vs. South Korea...,El Tri has advanced out of the group stage in ...
78,"Neymar, the Brazilian soccer star, will not be...",Neymar injured a lateral ligament on his right...


In [21]:
import xlsxwriter
filename = 'training_data_9_latest_sports.xlsx'
writer = pd.ExcelWriter(filename, engine='xlsxwriter')
dataframe.to_excel(writer, index=False)
writer.save()  